## **MSc Selected Topics in Data Science (Machine Learning) Final Project**

## **Student Name: Mohammad Awad**






---


---



####  Software defects datasets contain information about bugs in software modules (functions). Each dataset represents one software project with multiple modules. There are some datasets that are different releases for the same project. The features in the datasets describe the static code metrics in software modules where the output will be binary variable called defect (1= defected, 0=none-defected).



---


In [ ]:
#to silence warnings for clearer outputs
def warn(*args, **kwargs):
    pass
import warnings
warnings.warn = warn

#### Preprocessing

##### Importing libraries and creating a dictionary of csv files read from the given folder

In [ ]:
import pandas as pd
import numpy as np

In [ ]:
# Random state set for reproducibility purpouses
np.random.seed(711)

In [ ]:
flatFilesList=['ar1.csv',
 'ar3.csv',
 'ar4.csv',
 'ar5.csv',
 'ar6.csv',
 'cm1.csv',
 'pc1.csv',
 'pc2.csv',
 'pc3.csv',
 'pc4.csv']
flatFilesList


['ar1.csv',
 'ar3.csv',
 'ar4.csv',
 'ar5.csv',
 'ar6.csv',
 'cm1.csv',
 'pc1.csv',
 'pc2.csv',
 'pc3.csv',
 'pc4.csv']

In [ ]:
df={}
for file in flatFilesList:
  df[f"{file}"]=pd.read_csv(f'./{file}')
df.keys()

dict_keys(['ar1.csv', 'ar3.csv', 'ar4.csv', 'ar5.csv', 'ar6.csv', 'cm1.csv', 'pc1.csv', 'pc2.csv', 'pc3.csv', 'pc4.csv'])

In [ ]:
#Checking column names, number of records, if data contains any nulls and the data type for each column
for key in df.keys():
  print("\n" +"="*40)
  print(key)
  print("-"*40)
  print(df[key].info())


ar1.csv
----------------------------------------
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 121 entries, 0 to 120
Data columns (total 30 columns):
 #   Column                            Non-Null Count  Dtype  
---  ------                            --------------  -----  
 0   total_loc                         121 non-null    int64  
 1   blank_loc                         121 non-null    int64  
 2   comment_loc                       121 non-null    int64  
 3   code_and_comment_loc              121 non-null    int64  
 4   executable_loc                    121 non-null    int64  
 5   unique_operands                   121 non-null    int64  
 6   unique_operators                  121 non-null    int64  
 7   total_operands                    121 non-null    int64  
 8   total_operators                   121 non-null    int64  
 9   halstead_vocabulary               121 non-null    int64  
 10  halstead_length                   121 non-null    int64  
 11  halstead_volume      

In [ ]:
# Exploring data distribution to check for the need of normalization
for key in df.keys():
  print("\n" +"="*40)
  print(key)
  print("-"*40)
  print(df[key].head())


ar1.csv
----------------------------------------
   total_loc  blank_loc  comment_loc  code_and_comment_loc  executable_loc  \
0          7          0            4                     0               3   
1          9          0            1                     0               8   
2         21          0           14                     1               7   
3         30          0           11                     0              19   
4          8          0            2                     0               6   

   unique_operands  unique_operators  total_operands  total_operators  \
0                8                 6              10               12   
1                7                 8              15               20   
2               15                12              21               36   
3               16                18              50               70   
4                4                 5               5               10   

   halstead_vocabulary  ...  condition_cou

In [ ]:
# defects field has two classes in need of transforing into integer value
for key in df.keys():
  print("\n" +"="*40)
  print(key)
  print(df[key].defects.unique())


ar1.csv
[False  True]

ar3.csv
[ True False]

ar4.csv
[False  True]

ar5.csv
[False  True]

ar6.csv
[False  True]

cm1.csv
[False  True]

pc1.csv
[False  True]

pc2.csv
[False  True]

pc3.csv
[False  True]

pc4.csv
[False  True]


In [ ]:
# converting bool into integer and checking values
for key in df.keys():
  print("\n" +"="*40)
  print(key)
  df[key].defects=df[key].defects.astype('int')
  print(df[key].head(2))


ar1.csv
   total_loc  blank_loc  comment_loc  code_and_comment_loc  executable_loc  \
0          7          0            4                     0               3   
1          9          0            1                     0               8   

   unique_operands  unique_operators  total_operands  total_operators  \
0                8                 6              10               12   
1                7                 8              15               20   

   halstead_vocabulary  ...  condition_count  multiple_condition_count  \
0                   14  ...                0                         0   
1                   15  ...                0                         0   

   cyclomatic_complexity  cyclomatic_density  decision_density  \
0                      1                0.33               0.0   
1                      2                0.25               0.0   

   design_complexity  design_density  normalized_cyclomatic_complexity  \
0                  0             0.0    

In [ ]:
#Creating X, y variables for each dataset
df_Xy={}
for key in df.keys():
  X=df[key].drop('defects',axis=1)
  y=df[key].defects
  df_Xy[key]={'X':X,'y':y}
  print("\n" +"="*40)
  print(key,'\n X')
  print(df_Xy[key]['X'].head())
  print(key,'y')
  print(df_Xy[key]['y'].head())
  print("-"*40)


ar1.csv 
 X
   total_loc  blank_loc  comment_loc  code_and_comment_loc  executable_loc  \
0          7          0            4                     0               3   
1          9          0            1                     0               8   
2         21          0           14                     1               7   
3         30          0           11                     0              19   
4          8          0            2                     0               6   

   unique_operands  unique_operators  total_operands  total_operators  \
0                8                 6              10               12   
1                7                 8              15               20   
2               15                12              21               36   
3               16                18              50               70   
4                4                 5               5               10   

   halstead_vocabulary  ...  call_pairs  condition_count  \
0                  

In [ ]:
# Data needs normalization across features
from sklearn.preprocessing import MinMaxScaler
df_norm={}
for key in df.keys():
  col=df_Xy[key]['X'].columns
  scalar=MinMaxScaler()
  scaled=pd.DataFrame(scalar.fit_transform(df_Xy[key]['X']))
  scaled.columns=col
  df_norm[key]={'X':scaled, "y":df_Xy[key]['y']}
  print("\n" +"="*40)
  print(key)
  print(df_norm[key])



ar1.csv
{'X':      total_loc  blank_loc  comment_loc  code_and_comment_loc  executable_loc  \
0     0.053763   0.000000     0.133333                   0.0          0.0125   
1     0.075269   0.000000     0.033333                   0.0          0.0750   
2     0.204301   0.000000     0.466667                   0.5          0.0625   
3     0.301075   0.000000     0.366667                   0.0          0.2125   
4     0.064516   0.000000     0.066667                   0.0          0.0500   
..         ...        ...          ...                   ...             ...   
116   0.322581   0.000000     0.166667                   0.0          0.3125   
117   0.043011   0.000000     0.000000                   0.0          0.0500   
118   0.043011   0.000000     0.000000                   0.0          0.0500   
119   0.268817   0.000000     0.100000                   0.0          0.2750   
120   0.526882   0.222222     0.400000                   0.0          0.4375   

     unique_operands  un

In [ ]:
#to check class distribution

'''import seaborn as sns
import matplotlib.pyplot as plt
for key in df_norm.keys():
  plt.figure()
  plt.title(f'{key}')
  ax=sns.countplot(x='y',data=df_norm[key])
  ax.bar_label(ax.containers[0])
  print(ax,'\n','~'*25)'''
for key in df_norm.keys():
  print(key,'\n',df_norm[key]['y'].value_counts(),'='*25,'\n')

ar1.csv 
 0    112
1      9
Name: defects, dtype: int64 ========================= 

ar3.csv 
 0    55
1     8
Name: defects, dtype: int64 ========================= 

ar4.csv 
 0    87
1    20
Name: defects, dtype: int64 ========================= 

ar5.csv 
 0    28
1     8
Name: defects, dtype: int64 ========================= 

ar6.csv 
 0    86
1    15
Name: defects, dtype: int64 ========================= 

cm1.csv 
 0    449
1     49
Name: defects, dtype: int64 ========================= 

pc1.csv 
 0    959
1     77
Name: defects, dtype: int64 ========================= 

pc2.csv 
 0    916
1      4
Name: defects, dtype: int64 ========================= 

pc3.csv 
 0    683
1     70
Name: defects, dtype: int64 ========================= 

pc4.csv 
 0    693
1     87
Name: defects, dtype: int64 ========================= 



##### The numbers show that there is a great imbalance in the class distribution in all datasets





---





#### 1) Evaluate the performance of Support Vector Classifier and Logistic Regression, using 10-Folds cross validation on each single dataset? Describe the best parameters for both algorithms and what is the impact of other parameters? Do all datasets share the same configuration settings?

In [ ]:
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split, KFold, GridSearchCV
from sklearn.metrics import classification_report, confusion_matrix, f1_score

##### Building and evaluating the model using 10-Fold CV

In [ ]:
folds=KFold(n_splits=10, shuffle=True) #10 Folds CV
y_pred_SVC={}
y_pred_LR={}
y_actual={}
results={}
for key in df_norm.keys():
  X=df_norm[key]['X']
  y=df_norm[key]['y']
  fold=1
  for train_index, test_index in folds.split(X):
    X_train=X.loc[train_index]
    X_test=X.loc[test_index]
    y_train=y[train_index]
    y_test=y[test_index]

    ################### SVM
    param={'kernel':['linear', 'poly', 'rbf', 'sigmoid'],
           'C': np.logspace(-2,3,6)}

    SVC_model=GridSearchCV(SVC(),param)
    SVC_model.fit(X_train,y_train)

    y_hat_SVC=SVC_model.predict(X_test)
    y_pred_SVC[key]=y_hat_SVC
    
    ################### LOGISTIC REGRESSION
    parameters = {
    'penalty' : ['l1','l2','elasticnet', 'none'], 
    'C'       : np.logspace(-2,3,6),
    'solver'  : ['lbfgs', 'liblinear', 'newton-cg']}

    LR_model=GridSearchCV(LogisticRegression(),parameters)
    LR_model.fit(X_train,y_train)

    y_hat_LR=LR_model.predict(X_test)
    y_pred_LR[key]=y_hat_LR
    ###################
    y_actual[key]=y_test
    #print('Best Parameters SVC: ',SVC_model.best_params_ ,'   Accuracy: ',SVC_model.best_score_ ,'   classification_report',classification_report(y_actual[key], y_pred_SVC[key]),'\n'*4)
    #print('Best Parameters LR: ',LR_model.best_params_ ,'Accuracy: ',LR_model.best_score_ ,'classification_report',classification_report(y_actual[key], y_pred_LR[key]),'\n'*4)
    results[f'{key}{fold}']={"SVC":[SVC_model,f1_score(y_actual[key], y_pred_SVC[key])],"LR":[LR_model,f1_score(y_actual[key], y_pred_LR[key])]}
    fold=fold+1


##### CV model results were each key contains a dictionary with keys SVM and LR having a list of [model, f1-score]

The f1-score for the 1 class was considered to be the evaluation measure as it is our target of interest and since the data is imbalanced.

Summary of best models below, their best parameters can be extracted from results dictionary

In [ ]:
#ar1.csv all performed the same f1 =0 so no optimal values for both models
#ar3.csv6 both models have f1= 0.67  which was the best 
#ar4.csv2 both models have f1= 1 which was the best  
#ar5.csv5 both models have f1= 1 which was the best
#ar6.csv6 and ar6.csv10 both models have f1= 1 which was the best
#cm1.csv all performed the same f1 =0 so no optimal values for both models
#pc1.csv10 SVC has score of 0.40 while LR has 0
#pc2.csv all performed the same f1 =0 so no optimal values for both models
#pc3.csv10 LR performed best at 0.55 while SVC=0
#pc4.csv9 both models have f1= 0.67 which was the best

In [ ]:
for key in results.keys():
  for i in range(10):
      print(key,"F1-Measure for class 1",'\n')
      print('SVC',results[key]['SVC'][1])
      print('-'*60)
      print('LR',results[key]['LR'][1])
      print('='*60)

Streaming output truncated to the last 5000 lines.
LR 1.0
ar3.csv7 F1-Measure for class 1 

SVC 0.0
------------------------------------------------------------
LR 1.0
ar3.csv7 F1-Measure for class 1 

SVC 0.0
------------------------------------------------------------
LR 1.0
ar3.csv7 F1-Measure for class 1 

SVC 0.0
------------------------------------------------------------
LR 1.0
ar3.csv8 F1-Measure for class 1 

SVC 0.0
------------------------------------------------------------
LR 0.0
ar3.csv8 F1-Measure for class 1 

SVC 0.0
------------------------------------------------------------
LR 0.0
ar3.csv8 F1-Measure for class 1 

SVC 0.0
------------------------------------------------------------
LR 0.0
ar3.csv8 F1-Measure for class 1 

SVC 0.0
------------------------------------------------------------
LR 0.0
ar3.csv8 F1-Measure for class 1 

SVC 0.0
------------------------------------------------------------
LR 0.0
ar3.csv8 F1-Measure for class 1 

SVC 0.0
--------------------

##### The results above are very bad for classifiying defects (class 1)

In [ ]:
for key in results.keys():
    for i in results[key].keys():
        print(i)
        print(key,'\n')
        print('SVC   ',results[key]['SVC'][0].best_params_,results[key]['SVC'][0].best_score_)
        print('-'*40)
        print('LR    ',results[key]['LR'][0].best_params_,results[key]['LR'][0].best_score_)
        print('='*40)

SVC
ar1.csv1 

SVC    {'C': 0.01, 'kernel': 'linear'} 0.9354978354978355
----------------------------------------
LR     {'C': 0.01, 'penalty': 'l1', 'solver': 'liblinear'} 0.9354978354978355
LR
ar1.csv1 

SVC    {'C': 0.01, 'kernel': 'linear'} 0.9354978354978355
----------------------------------------
LR     {'C': 0.01, 'penalty': 'l1', 'solver': 'liblinear'} 0.9354978354978355
SVC
ar1.csv2 

SVC    {'C': 0.01, 'kernel': 'linear'} 0.9177489177489176
----------------------------------------
LR     {'C': 0.01, 'penalty': 'l1', 'solver': 'liblinear'} 0.9177489177489176
LR
ar1.csv2 

SVC    {'C': 0.01, 'kernel': 'linear'} 0.9177489177489176
----------------------------------------
LR     {'C': 0.01, 'penalty': 'l1', 'solver': 'liblinear'} 0.9177489177489176
SVC
ar1.csv3 

SVC    {'C': 0.01, 'kernel': 'linear'} 0.9268398268398268
----------------------------------------
LR     {'C': 0.01, 'penalty': 'l1', 'solver': 'liblinear'} 0.9268398268398268
LR
ar1.csv3 

SVC    {'C': 0.01, 'kernel':

In [ ]:
keys_best_param=['ar1.csv6','ar3.csv6','ar4.csv2','ar5.csv5','ar6.csv6','cm1.csv1','pc1.csv10','pc2.csv10','pc3.csv10', 'pc4.csv9']

In [ ]:
best_params_SVC={}
best_params_LR={}
newkey=list(df_norm.keys())
i=0
for key in keys_best_param:
  print(key,'\n')
  best_params_SVC[newkey[i]]=results[key]['SVC'][0].best_params_
  print('SVC   ',best_params_SVC[newkey[i]])
  print('-'*40)
  best_params_LR[newkey[i]]=results[key]['LR'][0].best_params_
  print('LR    ',best_params_LR[newkey[i]])
  print('='*40)
  i=i+1

ar1.csv6 

SVC    {'C': 0.01, 'kernel': 'linear'}
----------------------------------------
LR     {'C': 0.01, 'penalty': 'l1', 'solver': 'liblinear'}
ar3.csv6 

SVC    {'C': 1.0, 'kernel': 'linear'}
----------------------------------------
LR     {'C': 1.0, 'penalty': 'l2', 'solver': 'liblinear'}
ar4.csv2 

SVC    {'C': 0.1, 'kernel': 'poly'}
----------------------------------------
LR     {'C': 10.0, 'penalty': 'l2', 'solver': 'lbfgs'}
ar5.csv5 

SVC    {'C': 0.1, 'kernel': 'poly'}
----------------------------------------
LR     {'C': 100.0, 'penalty': 'l2', 'solver': 'liblinear'}
ar6.csv6 

SVC    {'C': 10.0, 'kernel': 'linear'}
----------------------------------------
LR     {'C': 100.0, 'penalty': 'l2', 'solver': 'lbfgs'}
cm1.csv1 

SVC    {'C': 0.01, 'kernel': 'linear'}
----------------------------------------
LR     {'C': 100.0, 'penalty': 'l1', 'solver': 'liblinear'}
pc1.csv10 

SVC    {'C': 10.0, 'kernel': 'rbf'}
----------------------------------------
LR     {'C': 0.01, 'pena

All datasets were run under the same conditions.

#### **2) What is the impact of class imbalance on your results, and which is the best evaluation measure should be used in this case?**

Class imbalance resulted in poor classification of our target class, were it became biased towards the detection of no defects and wouldn't detect the defects properly.

It is advised to use F1-Measure for evaluating the models as the accuracy is non representative in this case as most defects class are non defects.

#### **3) Which class imbalance learning technique could work efficiently with these datasets? Show your evidence from the experiments?**

SMOTE or random oversampling can be used to rebalance the data, ensamble methods like Random forest and XGBoost could also be used.

class_weight option in SVC could give good results.

In [ ]:
#using Random Forest
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import matthews_corrcoef
RF_result={}

folds=KFold(n_splits=10, shuffle=True) #10 Folds CV
grid = { 
    'n_estimators': [250,400],
    'max_depth' : [7,11,17,],
    'criterion' :['entropy'],
    'random_state' : [711]
}
RF_models=np.array([])
for key in df_norm.keys():
  X=df_norm[key]['X']
  y=df_norm[key]['y']
  for train_index, test_index in folds.split(X):
    X_train=X.loc[train_index]
    X_test=X.loc[test_index]
    y_train=y[train_index]
    y_test=y[test_index]
  
    RF = GridSearchCV(estimator=RandomForestClassifier(), param_grid=grid, cv= 3,scoring=matthews_corrcoef)
    RF.fit(X_train, y_train)

    #RF = RandomForestClassifier()
    #RF.fit(X_train, y_train)
    RF_predict = RF.predict(X_test)
    RF_result[key]={'y_actual':y_test,'y_pred':RF_predict}
    print('f1 score for RF and ',key,'=',f1_score(y_test,RF_predict))
    print('matthews_corrcoef for RF and ',key,'=',matthews_corrcoef(y_test,RF_predict))
    print(RF.best_params_,"----",RF.best_score_)
    RF_models=np.append(RF_models,RF)

f1 score for RF and  ar1.csv = 0.0
matthews_corrcoef for RF and  ar1.csv = 0.0
{'criterion': 'entropy', 'max_depth': 7, 'n_estimators': 250, 'random_state': 711} ---- nan
f1 score for RF and  ar1.csv = 0.0
matthews_corrcoef for RF and  ar1.csv = 0.0
{'criterion': 'entropy', 'max_depth': 7, 'n_estimators': 250, 'random_state': 711} ---- nan
f1 score for RF and  ar1.csv = 0.0
matthews_corrcoef for RF and  ar1.csv = 0.0
{'criterion': 'entropy', 'max_depth': 7, 'n_estimators': 250, 'random_state': 711} ---- nan
f1 score for RF and  ar1.csv = 0.0
matthews_corrcoef for RF and  ar1.csv = 0.0
{'criterion': 'entropy', 'max_depth': 7, 'n_estimators': 250, 'random_state': 711} ---- nan
f1 score for RF and  ar1.csv = 0.0
matthews_corrcoef for RF and  ar1.csv = 0.0
{'criterion': 'entropy', 'max_depth': 7, 'n_estimators': 250, 'random_state': 711} ---- nan
f1 score for RF and  ar1.csv = 0.0
matthews_corrcoef for RF and  ar1.csv = 0.0
{'criterion': 'entropy', 'max_depth': 7, 'n_estimators': 250, 'ran

In [ ]:
#Better results achieved but still not suffecient for most datasets

In [ ]:
#using svm balanced 
for key in df_norm.keys():
  X=df_norm[key]['X']
  y=df_norm[key]['y']
  param={'kernel':['linear', 'poly', 'rbf', 'sigmoid'],'C': np.logspace(-2,3,6),'class_weight':['balanced']}
  X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3)
  model = GridSearchCV(SVC(),param)
  model.fit(X, y)
  svc_pred=model.predict(X_test)
  print(key,'f1 score for ',key,'=',f1_score(y_test,svc_pred))
  print(key,'MCC score for ',key,'=',matthews_corrcoef(y_test,svc_pred))
  print(key,model.best_params_)


ar1.csv f1 score for  ar1.csv = 0.888888888888889
ar1.csv MCC score for  ar1.csv = 0.8807710121010885
ar1.csv {'C': 1.0, 'class_weight': 'balanced', 'kernel': 'poly'}
ar3.csv f1 score for  ar3.csv = 1.0
ar3.csv MCC score for  ar3.csv = 1.0
ar3.csv {'C': 0.01, 'class_weight': 'balanced', 'kernel': 'poly'}
ar4.csv f1 score for  ar4.csv = 0.6
ar4.csv MCC score for  ar4.csv = 0.609449400220044
ar4.csv {'C': 0.01, 'class_weight': 'balanced', 'kernel': 'poly'}
ar5.csv f1 score for  ar5.csv = 0.8571428571428571
ar5.csv MCC score for  ar5.csv = 0.8100925873009825
ar5.csv {'C': 1.0, 'class_weight': 'balanced', 'kernel': 'poly'}
ar6.csv f1 score for  ar6.csv = 0.6
ar6.csv MCC score for  ar6.csv = 0.6060915267313265
ar6.csv {'C': 1.0, 'class_weight': 'balanced', 'kernel': 'poly'}
cm1.csv f1 score for  cm1.csv = 0.11320754716981131
cm1.csv MCC score for  cm1.csv = 0.0
cm1.csv {'C': 0.01, 'class_weight': 'balanced', 'kernel': 'rbf'}
pc1.csv f1 score for  pc1.csv = 0.1142857142857143
pc1.csv MCC sco

In [ ]:
#The above models performed the best for most datasets

In [ ]:
#using RandomOverSampler
from imblearn.pipeline import Pipeline
from imblearn.over_sampling import  RandomOverSampler
from sklearn.metrics import matthews_corrcoef

folds=KFold(n_splits=10, shuffle=True) #10 Folds CV
yre_pred_SVC={}
yre_pred_LR={}
yre_actual={}
results_re={}
for key in df_norm.keys():
  X=df_norm[key]['X']
  y=df_norm[key]['y']

  fold=1
  for train_index, test_index in folds.split(X):
    X_train=X.loc[train_index]
    X_test=X.loc[test_index]
    y_train=y[train_index]
    y_test=y[test_index]
    X_train, y_train = RandomOverSampler().fit_resample(X_train, y_train)
    ################### SVM
    param={'kernel':['linear', 'poly', 'rbf', 'sigmoid'],
           'C': np.logspace(-2,3,6)}

    SVC_model=GridSearchCV(SVC(),param)
    SVC_model.fit(X_train,y_train)

    y_hat_SVC=SVC_model.predict(X_test)
    yre_pred_SVC[key]=y_hat_SVC
    
    ################### LOGISTIC REGRESSION
    parameters = {
    'penalty' : ['l1','l2','elasticnet', 'none'], 
    'C'       : np.logspace(-2,3,6),
    'solver'  : ['lbfgs', 'liblinear', 'newton-cg']}

    LR_model=GridSearchCV(LogisticRegression(),parameters)
    LR_model.fit(X_train,y_train)

    y_hat_LR=LR_model.predict(X_test)
    yre_pred_LR[key]=y_hat_LR
    ###################
    yre_actual[key]=y_test
    results_re[f'{key}{fold}']={"SVC":[SVC_model,f1_score(yre_actual[key], yre_pred_SVC[key])\
    ,matthews_corrcoef(yre_actual[key], yre_pred_SVC[key])],\
    "LR":[LR_model,f1_score(yre_actual[key], yre_pred_LR[key]),\
    matthews_corrcoef(yre_actual[key], yre_pred_LR[key])]}
    
    fold=fold+1


In [ ]:
results_re

{'ar1.csv1': {'SVC': [GridSearchCV(estimator=SVC(),
                param_grid={'C': array([1.e-02, 1.e-01, 1.e+00, 1.e+01, 1.e+02, 1.e+03]),
                            'kernel': ['linear', 'poly', 'rbf', 'sigmoid']}),
   0.0,
   0.0],
  'LR': [GridSearchCV(estimator=LogisticRegression(),
                param_grid={'C': array([1.e-02, 1.e-01, 1.e+00, 1.e+01, 1.e+02, 1.e+03]),
                            'penalty': ['l1', 'l2', 'elasticnet', 'none'],
                            'solver': ['lbfgs', 'liblinear', 'newton-cg']}),
   0.0,
   0.0]},
 'ar1.csv2': {'SVC': [GridSearchCV(estimator=SVC(),
                param_grid={'C': array([1.e-02, 1.e-01, 1.e+00, 1.e+01, 1.e+02, 1.e+03]),
                            'kernel': ['linear', 'poly', 'rbf', 'sigmoid']}),
   0.0,
   0.0],
  'LR': [GridSearchCV(estimator=LogisticRegression(),
                param_grid={'C': array([1.e-02, 1.e-01, 1.e+00, 1.e+01, 1.e+02, 1.e+03]),
                            'penalty': ['l1', 'l2', 'elasticnet', 'n

In [ ]:
for key in results_re.keys():
  print(key,'SVC',results_re[key]['SVC'][0].best_params_,results_re[key]['SVC'][2])
  print(key,'LR',results_re[key]['LR'][0].best_params_,results_re[key]['LR'][2])

ar1.csv1 SVC {'C': 1000.0, 'kernel': 'rbf'} 0.0
ar1.csv1 LR {'C': 0.01, 'penalty': 'none', 'solver': 'newton-cg'} 0.0
ar1.csv2 SVC {'C': 100.0, 'kernel': 'rbf'} 0.0
ar1.csv2 LR {'C': 0.01, 'penalty': 'none', 'solver': 'newton-cg'} 0.0
ar1.csv3 SVC {'C': 10.0, 'kernel': 'rbf'} -0.17407765595569785
ar1.csv3 LR {'C': 0.01, 'penalty': 'none', 'solver': 'lbfgs'} -0.21320071635561041
ar1.csv4 SVC {'C': 100.0, 'kernel': 'rbf'} -0.09090909090909091
ar1.csv4 LR {'C': 100.0, 'penalty': 'l1', 'solver': 'liblinear'} -0.13483997249264842
ar1.csv5 SVC {'C': 100.0, 'kernel': 'poly'} 0.0
ar1.csv5 LR {'C': 0.01, 'penalty': 'none', 'solver': 'lbfgs'} 0.0
ar1.csv6 SVC {'C': 1000.0, 'kernel': 'linear'} 0.0
ar1.csv6 LR {'C': 0.01, 'penalty': 'none', 'solver': 'newton-cg'} 0.0
ar1.csv7 SVC {'C': 100.0, 'kernel': 'rbf'} 0.0
ar1.csv7 LR {'C': 0.01, 'penalty': 'none', 'solver': 'newton-cg'} 0.0
ar1.csv8 SVC {'C': 100.0, 'kernel': 'poly'} 0.674199862463242
ar1.csv8 LR {'C': 0.01, 'penalty': 'none', 'solver': 'l

In [ ]:
#Better performance in comparison to no balancing

#### **4) Is there improvement when apply feature selection? Which features do you think are the most predictive ones? Explain the way you have decided to select the features.**

The selection of features was done by trail and error changing the values of k for selectKbest and evaluating the average f1 measure as below.

a correlation matrix could also be done to exclude the highly correlated features (colinearity)

In [ ]:
from sklearn.feature_selection  import SelectKBest 
from sklearn.feature_selection import mutual_info_classif

reduced_score={}
best_features={}
for key in df_norm.keys():
  X=df_norm[key]['X']
  y=df_norm[key]['y']
  scorer=[]
  for train_index, test_index in folds.split(X):
    features= X.columns
    features
    fs=SelectKBest(k=15, score_func=mutual_info_classif)
    fs.fit(X,y)
    mask=fs.get_support()
    names=features[mask]
    names
    X_reduced=X[names]
    X_reduced
    X_train, X_test, y_train, y_test = train_test_split(X_reduced, y, test_size=0.3)

    model = SVC(C=best_params_SVC[key]['C'],kernel=best_params_SVC[key]['kernel'], class_weight='balanced')
    model.fit(X_train, y_train)
    svc_pred=model.predict(X_test)
    scorer=np.append(scorer,f1_score(y_test,svc_pred))
    reduced_score[key]=[scorer]
    best_features[key]=names

In [ ]:
for key in reduced_score.keys():
  print(f'f1 score for file {key}= ' ,np.mean(reduced_score[key]))

f1 score for file ar1.csv=  0.28137494021999043
f1 score for file ar3.csv=  0.5126190476190475
f1 score for file ar4.csv=  0.4832972582972583
f1 score for file ar5.csv=  0.7171428571428572
f1 score for file ar6.csv=  0.3373236512942395
f1 score for file cm1.csv=  0.11769215396169974
f1 score for file pc1.csv=  0.2661293373283614
f1 score for file pc2.csv=  0.03181129199307888
f1 score for file pc3.csv=  0.0626914962775876
f1 score for file pc4.csv=  0.4476426715552509


Yes feature selection improved the results. the best features were:

In [ ]:
best_features

{'ar1.csv': Index(['executable_loc', 'unique_operators', 'total_operands',
        'total_operators', 'halstead_vocabulary', 'halstead_length',
        'halstead_volume', 'halstead_level', 'halstead_difficulty',
        'halstead_effort', 'halstead_error', 'halstead_time', 'branch_count',
        'cyclomatic_complexity', 'normalized_cyclomatic_complexity'],
       dtype='object'),
 'ar3.csv': Index(['total_loc', 'blank_loc', 'code_and_comment_loc', 'executable_loc',
        'unique_operands', 'total_operands', 'total_operators',
        'halstead_vocabulary', 'halstead_length', 'halstead_volume',
        'halstead_error', 'branch_count', 'decision_count', 'condition_count',
        'cyclomatic_complexity'],
       dtype='object'),
 'ar4.csv': Index(['total_loc', 'blank_loc', 'comment_loc', 'executable_loc',
        'unique_operands', 'unique_operators', 'total_operands',
        'total_operators', 'halstead_vocabulary', 'halstead_length',
        'halstead_volume', 'halstead_error', 'b

#### **5) For Datasets with multiple versions, does the accuracy improve when we use pre-release as training and pos release as testing?**

In [ ]:
from sklearn.metrics import matthews_corrcoef
#for ar projects
X_train=pd.concat([df_norm['ar1.csv']["X"],df_norm['ar3.csv']["X"],df_norm['ar4.csv']["X"],df_norm['ar5.csv']["X"]], axis=0)
y_train=pd.concat([df_norm['ar1.csv']['y'],df_norm['ar3.csv']['y'],df_norm['ar4.csv']['y'],df_norm['ar5.csv']['y']], axis=0)

X_test=df_norm['ar6.csv']['X']
y_test=df_norm['ar6.csv']['y']

#Model training and evaluation
################### SVM
param={'kernel':['linear', 'poly', 'rbf', 'sigmoid'],
        'C': np.logspace(-3,2,6)}

k=10
SVC_model=GridSearchCV(SVC(),param,cv=k,scoring=matthews_corrcoef)
SVC_model.fit(X_train,y_train)

y_hat_SVC=SVC_model.predict(X_test)
y_pred_SVC=y_hat_SVC

print('SVC   ',classification_report(y_test,y_pred_SVC),matthews_corrcoef(y_test,y_pred_SVC))
print('SVC   ',SVC_model.best_params_)

################### LOGISTIC REGRESSION
parameters = {
'penalty' : ['l1','l2','elasticnet', 'none'], 
'C'       : np.logspace(-5,5,11),
'solver'  : ['lbfgs', 'liblinear', 'newton-cg']}

LR_model=GridSearchCV(LogisticRegression(),parameters,cv=k)
LR_model.fit(X_train,y_train)

y_hat_LR=LR_model.predict(X_test)
y_pred_LR=y_hat_LR
print('LR   ',classification_report(y_test,y_pred_LR),matthews_corrcoef(y_test,y_pred_LR))
print('LR   ',LR_model.best_params_)



SVC                  precision    recall  f1-score   support

           0       0.85      1.00      0.92        86
           1       0.00      0.00      0.00        15

    accuracy                           0.85       101
   macro avg       0.43      0.50      0.46       101
weighted avg       0.73      0.85      0.78       101
 0.0
SVC    {'C': 0.001, 'kernel': 'linear'}
LR                  precision    recall  f1-score   support

           0       0.87      1.00      0.93        86
           1       1.00      0.13      0.24        15

    accuracy                           0.87       101
   macro avg       0.93      0.57      0.58       101
weighted avg       0.89      0.87      0.83       101
 0.3403305978382135
LR    {'C': 1.0, 'penalty': 'l1', 'solver': 'liblinear'}


In [ ]:
#Not much of an improvement in comparison to separate datasets

#### **6) what are the necessary steps that you should take care when avoiding overfitting for both logistic regression and SVC?**

You can apply regularization to logistic regression and avoid very high order polynomials, and increase the margin for SVC by allowing for more slack variables by adjusting the value of C and using the appropriate kernals, Data cleaning/preprocessing including balancing unbalanced data, scaling and feature selection help avoid overfitting, which the above questions touched upon. Using CV is great way to evaluate if the model is overfitting as it tests it on all portions of the data as unseen in every iteration.

